In [1]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
import json

In [2]:
from helper import *

In [3]:
# connect to the mongo client
client = connect_to_mongo()

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [4]:
# get the collection storing the problems
collection = get_collection(client = client,
    database_name = 'qa-repo',
    collection_name = 'qa')

# collection storing key : value
# key is 'problem_name'
# the value is the json containing all test cases

In [7]:
# get all problems' names from the collection
problems = set()
for problem in collection.find():
    problems.add(problem['problem_name'])

print(problems)

{'valid_palindrome_3', 'maximum_subarray', 'valid_palindrome', 'koko_eating_bananas', 'valid_palindrome_2', 'binary_search', 'contains_duplicate', 'longest_turbulent_subarray', 'two_sum', 'valid_parentheses'}


In [6]:
# Get the current directory where the script is located
current_directory = os.getcwd()

# Define the database directory (change this to your actual directory name)
database_directory = "test_cases_and_answers"

# Construct the full path to the database directory
database_path = os.path.join(current_directory, database_directory)

# Check if the directory exists
if os.path.exists(database_path) and os.path.isdir(database_path):

    # List all files in the directory
    files = os.listdir(database_path)
    
    # Print the list of files
    for file_name in files:

        file_path = os.path.join(database_path, file_name)
        print(file_name)

        problem_name = file_name.replace('.json', '')

        if os.path.isfile(file_path):

            if problem_name not in problems:

                try:

                    with open(file_path, "r") as file:
                        
                        data = json.load(file)
                        data['problem_name'] = problem_name
                        inserted_object = collection.insert_one(data)
                        print(f"Uploaded file: {file_path}, Object ID: {inserted_object.inserted_id}")

                except Exception as e:
                    print(f"Error uploading file {file_path}: {str(e)}")

else:
    print(f"Database directory '{database_path}' does not exist.")

valid_parentheses.json
Uploaded file: /Users/nnt/Documents/Developer/Advanced Software Design/AlgorithmArena/Q&A/test_cases_and_answers/valid_parentheses.json, Object ID: 65473db109e320bc340ea7aa
maximum_subarray.json
Uploaded file: /Users/nnt/Documents/Developer/Advanced Software Design/AlgorithmArena/Q&A/test_cases_and_answers/maximum_subarray.json, Object ID: 65473db109e320bc340ea7ab
two_sum.json
Uploaded file: /Users/nnt/Documents/Developer/Advanced Software Design/AlgorithmArena/Q&A/test_cases_and_answers/two_sum.json, Object ID: 65473db209e320bc340ea7ac
binary_search.json
valid_palindrome_3.json
Uploaded file: /Users/nnt/Documents/Developer/Advanced Software Design/AlgorithmArena/Q&A/test_cases_and_answers/valid_palindrome_3.json, Object ID: 65473db209e320bc340ea7ad
valid_palindrome_2.json
Uploaded file: /Users/nnt/Documents/Developer/Advanced Software Design/AlgorithmArena/Q&A/test_cases_and_answers/valid_palindrome_2.json, Object ID: 65473db209e320bc340ea7ae
longest_turbulent_s